# 3W dataset's General Presentation

This is a general presentation of the 3W dataset, to the best of its authors' knowledge, the first realistic and public dataset with rare undesirable real events in oil wells that can be readily used as a benchmark dataset for development of machine learning techniques related to inherent difficulties of actual data.

For more information about the theory behind this dataset, refer to the paper **A Realistic and Public Dataset with Rare Undesirable Real Events in Oil Wells** published in the **Journal of Petroleum Science and Engineering** (link [here](https://doi.org/10.1016/j.petrol.2019.106223)).

# 1. Introduction

This Jupyter Notebook presents the 3W dataset in a general way. For this, some tables, graphs, and statistics are presented.

# 2. Imports and Configurations

In [43]:
import sys
import os
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
import bisect
import sklearn
import sklearn.model_selection
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, multilabel_confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# 3. Instances' Structure

Below, all 3W dataset's instances are loaded and the first one of each knowledge source (real, simulated and hand-drawn) is partially displayed.

In [2]:
class d3w():
    '''
    Class for managing Petrobras 3W dataset
    '''
    def __init__(self, path3w):
        self.path3w = path3w
        self.df = self.__load_df()
        return

    def __load_df(self):

        d = dict()
        d['origin'] = []
        d['label'] = []
        d['path'] = []
        d['nlines'] = []
        for i in pathlib.Path(self.path3w).iterdir():
            if i.stem.isnumeric():
                print(i)
                label = int(i.stem)
                for fp in i.iterdir():
                    # Considers only csv files
                    if fp.suffix == ".csv":

                        if (fp.stem.startswith("SIMULATED")):
                            d['origin'].append('S')
                        elif fp.stem.startswith("DRAWN"):
                            d['origin'].append('D')
                        else:
                            d['origin'].append('R')
                        
                        d['label'].append(label)
                        d['path'].append(fp)
                        d['nlines'].append(self.file_len(fp)-1)
                        
        return pd.DataFrame(d)
    
    def split(self, real=True, simul=True, drawn=True, test_size=0.2, val_size=0.1, sample_n=None):
        
        tmp0_df = self.get_df(real, simul, drawn)
        
        if sample_n is not None:
            N = len(tmp0_df.index)
            if N > sample_n:
                ds_list = []
                for i, ni in tmp0_df.groupby('label').count().nlines.items():
                    ns = ni*sample_n//N
                    ds_list.append(tmp0_df[tmp0_df.label == i].sample(n=ns, random_state=200560))
                tmp0_df = pd.concat(ds_list)            
        
        tmp_df, test_df = sklearn.model_selection.train_test_split(tmp0_df, 
                                                        test_size=test_size, 
                                                        random_state=200560, 
                                                        shuffle=True, 
                                                        stratify=tmp0_df['label'])
        
        if val_size == 0:
            print('Instances Train: {}  Test: {}'.format(len(tmp_df.index), 
                                                         len(test_df.index)))
            return tmp_df.reset_index(drop=True),\
                   test_df.reset_index(drop=True)
        
        train_df, val_df = sklearn.model_selection.train_test_split(tmp_df, test_size=val_size, 
                                                        random_state=200560, 
                                                        shuffle=True, 
                                                        stratify=tmp_df['label'])
        print('Instances Train: {}  Test: {}  Validation: {}'.format(len(train_df.index), 
                                                                     len(test_df.index), 
                                                                     len(val_df.index)))
        
        return train_df.reset_index(drop=True),\
               test_df.reset_index(drop=True),\
               val_df.reset_index(drop=True)
    
    def file_len(self, filename):
        j = 0
        with open(filename) as f:
            for i, x in enumerate(f):
                if x.strip() == '':
                    j += 1
        return i + 1 - j
    
    def get_df(self, real=True, simul=True, drawn=True):
        sel = []
        if real:
            sel.append('R')
        if simul:
            sel.append('S')
        if drawn:
            sel.append('D')
        if sel:
            return self.df[self.df['origin'].isin(sel)].drop(columns=['origin']).reset_index(drop=True)
    
    @property
    def all(self):
        return self.df.drop(columns=['origin'])
    @property
    def real(self):
        return self.df[self.df['origin']=='R'].drop(columns=['origin']).reset_index(drop=True)
    @property
    def simul(self):
        return self.df[self.df['origin']=='S'].drop(columns=['origin']).reset_index(drop=True)
    @property
    def drawn(self):
        return self.df[self.df['origin']=='D'].drop(columns=['origin']).reset_index(drop=True)


In [3]:
if pathlib.Path('dset_sgd.pkl').exists():
  with open('dset_sgd.pkl', 'rb') as f:
    dset = pickle.load(f)
else:
    dset = d3w('../dataset')
    with open('dset_sgd.pkl', 'wb') as f:
      pickle.dump(dset, f)

In [39]:
flist0 = ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'P-JUS-CKGL', 'T-JUS-CKGL', 'QGL']
categories=[0,1,2,3,4,5,6,7,8,101,102,103,104,105,106,107,108]

train_df, test_df, val_df = dset.split(real=True, simul=True, drawn=True, test_size=0.2, val_size=0.15, sample_n=300)

Instances Train: 201  Test: 60  Validation: 36


Each instance is stored in a CSV file and loaded into a pandas DataFrame. Each observation is stored in a line in the CSV file and loaded as a line in the pandas DataFrame. The first line of each CSV file contains a header with column identifiers. Each column of CSV files stores the following type of information:

* **timestamp**: observations timestamps loaded into pandas DataFrame as its index;
* **P-PDG**: pressure variable at the Permanent Downhole Gauge (PDG);
* **P-TPT**: pressure variable at the Temperature and Pressure Transducer (TPT);
* **T-TPT**: temperature variable at the Temperature and Pressure Transducer (TPT);
* **P-MON-CKP**: pressure variable upstream of the production choke (CKP);
* **T-JUS-CKP**: temperature variable downstream of the production choke (CKP);
* **P-JUS-CKGL**: pressure variable upstream of the gas lift choke (CKGL);
* **T-JUS-CKGL**: temperature variable upstream of the gas lift choke (CKGL);
* **QGL**: gas lift flow rate;
* **class**: observations labels associated with three types of periods (normal, fault transient, and faulty steady state).

Other information are also loaded into each pandas Dataframe:

* **label**: instance label (event type);
* **well**: well name. Hand-drawn and simulated instances have fixed names. Real instances have names masked with incremental id;
* **id**: instance identifier. Hand-drawn and simulated instances have incremental id. Each real instance has an id generated from its first timestamp.

More information about these variables can be obtained from the following publicly available documents:

* ***Option in Portuguese***: R.E.V. Vargas. Base de dados e benchmarks para prognóstico de anomalias em sistemas de elevação de petróleo. Universidade Federal do Espírito Santo. Doctoral thesis. 2019. https://github.com/ricardovvargas/3w_dataset/raw/master/docs/doctoral_thesis_ricardo_vargas.pdf.
* ***Option in English***: B.G. Carvalho. Evaluating machine learning techniques for detection of flow instability events in offshore oil wells. Universidade Federal do Espírito Santo. Master's degree dissertation. 2021. https://github.com/ricardovvargas/3w_dataset/raw/master/docs/master_degree_dissertation_bruno_carvalho.pdf.

# Preprocessing

The following table shows the amount of instances that compose the 3W dataset, by knowledge source (real, simulated and hand-drawn instances) and by instance label.

In [5]:
class CustomDataGen():
    '''
    Generator for Keras models of 3W dataset based on:
    https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3
    
    '''
    
    def __init__(self, df, X_col, y_col, categories,
                 batch_size,
                 seq_length=15,
                 tmp_path='/tmp'):
        
        self.df = df.copy().reset_index()
        self.X_col = X_col
        self.y_col = y_col
        self.categories = categories
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.tmp_path = pathlib.Path(tmp_path)
        self.file = None
        self.dt = None
        
        self.n = self.__calc_n()
    
    def __calc_n(self):
        self.df['nbatches'] = np.int32(np.ceil((np.ceil(self.df['nlines'] / 60)-self.seq_length+1)/self.batch_size))
        self.df['ibatch'] = self.df['nbatches'].cumsum() - 1
        return int(self.df['nbatches'].sum())
    
    def plot(self, ifile):
        
        ds = self.__get_ds(self.df['path'][ifile], Norm=False)
        
        fig, axs = plt.subplots(nrows=len(self.X_col)+1, figsize=(10, 12), sharex=True)
        
        fig.suptitle(self.df['path'][ifile])

        for i, vs in enumerate(self.X_col):
            axs[i].plot(ds.index, ds[(vs, 'mean')])
            axs[i].fill_between(ds.index, ds[(vs, 'mean')]-1.96*ds[(vs, 'std')], 
                            ds[(vs, 'mean')]+1.96*ds[(vs, 'std')], 
                            alpha=0.2)
            axs[i].set_ylabel(vs)
            axs[i].grid()
        
        id = np.argsort(ds[(self.y_col, 'mode')])
        
        axs[i+1].scatter([ds.index[i] for i in id], [str(ds[(self.y_col, 'mode')][i]) for i in id], marker='.')
        
        axs[i+1].set_ylabel(self.y_col)
        
        axs[i+1].set_xlabel('minute')

        plt.show()    
    
    def on_epoch_end(self):
            pass
    
    def pkl_path(self, p):
        new_p = self.tmp_path.joinpath(p.parts[-2], p.stem+'.pkl')
        new_p.parent.mkdir(exist_ok=True, parents=True)
        return new_p
    
    def __get_ds(self, p, Norm=True):
    
        pkl_p = self.pkl_path(p)
        
        if pkl_p.exists():
            ds = pd.read_pickle(pkl_p)
        else:
        
            dfo = pd.read_csv(p, index_col="timestamp", parse_dates=["timestamp"])

            if np.any(dfo[self.y_col].isna()):
                dfo[self.y_col] = dfo[self.y_col].fillna(method='ffill')
            dfo[self.y_col] = dfo[self.y_col].astype('int')

            flist = []
            flist0 = []
            for f in self.X_col:
                nas = np.sum(dfo[f].isna())
                if nas > 0:
                    if nas < len(dfo.index) * 0.2:
                        dfo[f] = dfo[f].fillna(method='ffill')
                        flist.append(f)
                    else:
                        flist0.append(f)
                else:
                    flist.append(f)

            fdict=dict()
            for f in flist:
                fdict[f] = ['mean','std']

            def mode(series):
                return pd.Series.mode(series)[0]

            fdict[self.y_col] = [mode]

            dfo['minute'] = (dfo.index-dfo.index[0])//np.timedelta64(1,'m')

            ds = dfo.groupby('minute').agg(fdict)

            #ds = ds.iloc[:-1]

            for f in flist0:
                ds[f, 'mean'] = np.NaN
                ds[f, 'std'] = np.NaN
                
            ds.to_pickle(pkl_p)
        
        if Norm:
            ds = self.__Norm(ds)
            if ds.isnull().any().any():
                print(p, pkl_p)
        
        return ds[self.X_col + [self.y_col]]
        
    def __Norm(self, ds, nas_v=0):
        dn = ds.fillna(value=nas_v)
        sc = sklearn.preprocessing.StandardScaler()
        dn = pd.DataFrame(sc.fit_transform(dn.values), 
                                           index=dn.index, 
                                           columns=dn.columns)
        dn[(self.y_col, 'mode')] = ds[(self.y_col, 'mode')]
        return dn
        
    def __get_output(self, y):
        
        ohe = sklearn.preprocessing.OneHotEncoder(categories=self.categories, sparse= False)
        
        return ohe.fit_transform(y)
    
    def __get_data(self, i, j, p):
        # Generates data containing batch_size samples

        if p != self.file:
            self.ts = self.__get_ds(p)
            self.file = p

        nf = len(self.X_col)
        mat = np.zeros(shape=(self.batch_size, 2*nf*self.seq_length))
        
        for k in range(self.batch_size):
            if self.seq_length+k+j*self.batch_size > len(self.ts.index):
                mat = mat[:k]
                break
            mat[k] = np.reshape(self.ts[k+j*self.batch_size:self.seq_length+k+j*self.batch_size][self.X_col].to_numpy(), (1, -1))
        return mat, self.ts.iloc[j*self.batch_size+self.seq_length-1:j*self.batch_size+self.seq_length+self.batch_size-1][(self.y_col, 'mode')].to_numpy()
    
    def __getitem__(self, index):
        
        i = bisect.bisect_left(self.df.ibatch, index)
        if i > 0:
            j = index - self.df.ibatch[i-1] - 1
        else:
            j = index
        
        p = self.df.path[i]       
        
        # print(index, i, j, p)
        
        X, y = self.__get_data(i, j, p)        
        
        return X, y #self.__get_output(y)
    
    def __len__(self):
        return self.n
    
    def get_y(self):
        y = np.empty(0, 'int')
        for p in self.df['path']:
            pkl_p = self.pkl_path(p)
            if pkl_p.exists():
                ds = pd.read_pickle(pkl_p)
            else:
                ds = self.__get_ds(p, Norm = False)
            y = np.append(y, ds[self.y_col].iloc[self.seq_length-1:].astype('int'))
        return y

In [269]:
train = CustomDataGen(train_df, flist0, 'class', categories, 64, 30, 'D:/datatmp')

In [ ]:
ys = []
for i in range(train.__len__()):
    X, y = train.__getitem__(i)
    ys += list(y)

In [31]:
yss = train.get_y()
w = dict(zip(np.unique(yss), compute_class_weight('balanced', classes=np.unique(yss), y=yss)))
for j in categories:
    if j not in d.keys():
        w[j] = 1.0

{0: 0.33873615100533444,
 1: 1.0482539682539682,
 2: 44.026666666666664,
 3: 0.6438529784537389,
 4: 1.7517241379310344,
 5: 0.32073822243807676,
 6: 0.8466666666666667,
 7: 7.178260869565217,
 8: 7.23329682365827,
 101: 0.5615646258503402,
 102: 24.3690036900369,
 105: 1.6166462668298653,
 106: 2.1166666666666667,
 107: 1.0414761078694212,
 108: 2.0091268634012778,
 103: 1.0,
 104: 1.0}

# Model

In [61]:
%%time

clf = OneVsRestClassifier(SGDClassifier(loss='perceptron', alpha=0.01, penalty='elasticnet', l1_ratio=0., class_weight=w), n_jobs=-1)
#clf = SGDClassifier(penalty='elasticnet', l1_ratio=0.75, class_weight=w, n_jobs=-1)
#clf = PassiveAggressiveClassifier(C=10, class_weight=w, n_jobs=-1)
train = CustomDataGen(train_df, flist0, 'class', categories, 128, 30, 'D:/datatmp')
val = CustomDataGen(val_df, flist0, 'class', categories, 128, 30, 'D:/datatmp')
n = train.__len__()
n_epochs = 10
for iter in range(n_epochs):
    train = CustomDataGen(train_df, flist0, 'class', categories, 128, 30, 'D:/datatmp')
    for i in range(n):
        print('\r' + str(iter+1) + ': ' + str(i+1) +  ' / ' + str(n), end='    ', flush=True)
        X, y = train.__getitem__(i)
        clf.partial_fit(X, y, classes=categories)
    ys = []
    ypreds = []
    for i in range(val.__len__()):
        X, y = val.__getitem__(i)
        ypreds += list(clf.predict(X))
        ys += list(y)
    print('\n   val f1 score macro: ', round(f1_score(ys, ypreds, zero_division=1, average='macro'), 3), '\n')
    
    train_df = train_df.sample(frac=1, random_state=200560).reset_index(drop=True)
print('\n')

1: 784 / 784    
   val f1 score macro:  0.017 

2: 784 / 784    
   val f1 score macro:  0.02 

3: 784 / 784        
   val f1 score macro:  0.017 

4: 361 / 784    

KeyboardInterrupt: 

In [37]:
%%time

test = CustomDataGen(test_df, flist0, 'class', categories, 64, 30, 'D:/datatmp')
ys = []
ypreds = []
n = test.__len__()
for i in range(n):
    print('\r' + str(i+1) +  ' / ' + str(n), end='', flush=True)
    X, y = test.__getitem__(i)
    ypreds += list(clf.predict(X))
    ys += list(y)
print('\n')
print(classification_report(ys, ypreds, zero_division=True))

394 / 394

              precision    recall  f1-score   support

           0       0.15      0.16      0.15      4827
           1       0.72      0.01      0.03      1347
           2       1.00      0.00      0.00         9
           3       0.41      0.13      0.20      2195
           4       0.06      0.57      0.11       996
           5       0.42      0.32      0.36      5654
           6       0.00      0.00      0.00      2100
           8       0.09      0.10      0.09       299
         101       0.00      0.00      0.00      2918
         102       0.01      0.03      0.01        97
         103       0.00      1.00      0.00         0
         104       0.00      1.00      0.00         0
         105       0.84      0.45      0.59       786
         106       0.45      0.36      0.40       840
         108       0.04      0.04      0.04       541

    accuracy                           0.19     22609
   macro avg       0.28      0.28      0.13     22609
weighted avg   

In [38]:
%%time

ys = []
ypreds = []
n = train.__len__()
for i in range(n):
    print('\r' + str(i+1) +  ' / ' + str(n), end='', flush=True)
    X, y = train.__getitem__(i)
    ypreds += list(clf.predict(X))
    ys += list(y)
print('\n')
print(classification_report(ys, ypreds, zero_division=True))

1700 / 1700

              precision    recall  f1-score   support

           0       0.16      0.14      0.14     19496
           1       0.64      0.01      0.01      6300
           2       1.00      0.00      0.00       150
           3       0.39      0.10      0.16     10257
           4       0.04      0.52      0.08      3770
           5       0.32      0.34      0.33     20590
           6       0.00      0.00      0.00      7800
           7       1.00      0.00      0.00       920
           8       0.12      0.28      0.16       913
         101       0.00      0.00      0.00     11760
         102       0.04      0.20      0.07       271
         103       0.00      1.00      0.00         0
         104       0.00      1.00      0.00         0
         105       0.74      0.29      0.42      4085
         106       0.36      0.34      0.35      3120
         107       0.00      0.00      0.00      6341
         108       0.03      0.03      0.03      3287

    accuracy 

In [16]:
clf.classes_

array([  0,   1,   2,   3,   4,   5,   6,   7,   8, 101, 102, 103, 104,
       105, 106, 107, 108])